In [4]:
#https://colab.research.google.com/drive/1kAlGGGsZaFaFoL0lZ0HK4xUR6QS8gipn#scrollTo=svum1ZL4dlHn
import wget

In [13]:
url = 'http://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/08/germeval2019GoldLabelsSubtask1_2.txt'
filename = wget.download(url)
print(filename)
url2 = "http://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/09/germeval2019.training_subtask1_2_korrigiert.txt"
filename2 = wget.download(url2)
print(filename2)

100% [............................................................................] 543975 / 543975germeval2019GoldLabelsSubtask1_2 (2).txt
100% [............................................................................] 697779 / 697779germeval2019.training_subtask1_2_korrigiert.txt


In [14]:
import pandas as pd
class_list = ['INSULT','ABUSE','PROFANITY','OTHER']
df1 = pd.read_csv('germeval2019GoldLabelsSubtask1_2.txt',sep='\t', 
                  lineterminator='\n',encoding='utf8',names=["tweet", "task1", "task2"])
df2 = pd.read_csv('germeval2019.training_subtask1_2_korrigiert.txt',sep='\t', 
                  lineterminator='\n',encoding='utf8',names=["tweet", "task1", "task2"])
df = pd.concat([df1,df2])
df['task2'] = df['task2'].str.replace('\r', "")
df['pred_class'] = df.apply(lambda x:  class_list.index(x['task2']),axis=1)
df = df[['tweet','pred_class']]
print(df.shape)
df.head()

(7011, 2)


,tweet,pred_class
0,@JanZimmHHB @mopo Komisch das die Realitätsver...,0
1,@faznet @Gruene_Europa @SPDEuropa @CDU CDU ste...,1
2,"@DLFNachrichten Die Gesichter, Namen, Religion...",3
3,@welt Wie verwirrt muss man sein um sich zu we...,1
4,@hacker_1991 @torben_braga Weil die AfD den Fe...,1


In [15]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10)
print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (6309, 2)
test shape:  (702, 2)


In [22]:
from simpletransformers.classification import ClassificationModel
# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}
# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-german-cased",
    num_labels=4,
    args=train_args,
    use_cuda=False
)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [23]:
model.train_model(train_df)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)
result

In [ ]:
import os
import tarfile
def save_model(model_path='',file_name=''):
    files = [files for root, dirs, files in os.walk(model_path)][0]
    with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
        for file in files:
            f.add(f'{model_path}/{file}')

In [ ]:
save_model('outputs','germeval-bert-german')

In [ ]:
!tar -zxvf ./germeval-bert-german.tar.gz

In [ ]:
!rm -rf outputs

In [ ]:
import os
import tarfile
def unpack_model(model_name=''): 
    tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
    tar.extractall()
    tar.close()
unpack_model('germeval-bert-german')

In [ ]:
from simpletransformers.classification import ClassificationModel
# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}
# Create a ClassificationModel
model = ClassificationModel(
    "bert", "outputs/",
    num_labels=4,
    args=train_args
)

In [ ]:
class_list = ['INSULT','ABUSE','PROFANITY','OTHER']

test_tweet = "Du blöde Kuh!"

predictions, raw_outputs = model.predict([test_tweet])

print(class_list[predictions[0]])

In [ ]:
class_list = ['INSULT','ABUSE','PROFANITY','OTHER']

test_tweet = "Frau #Böttinger meine Meinung dazu ist sie sollten uns mit ihrem Pferdegebiss nicht weiter belästigen #WDR"

predictions, raw_outputs = model.predict([test_tweet])

print(class_list[predictions[0]])